In [1]:
import sys;
import os;
import glob;
import math;
import numpy as np;
import glob;
import random;
import time;
import torch;
import torch.optim as optim;
import torch.nn as nn;
import json
sys.path.append(os.getcwd());
sys.path.append('../../');
sys.path.append(os.path.abspath('../../../src/'));
# sys.path.append(os.path.join(os.getcwd(), 'torch/resources'));
import common.utils as U;
import common.opts as opts;
# import resources.models as models;
import th.resources.calculator as calc;
import common.tlopts as tlopts
# import resources.train_generator as train_generator;
import argparse
from itertools import repeat
import pathlib

In [2]:
from SharedLibs.config_utility import *

In [4]:
one_sec_input_len = 20150;

In [ ]:
# def make_layers(in_channels, out_channels, kernel_size, stride=(1,1), padding=0, bias=False):
#     conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias);
#     nn.init.kaiming_normal_(conv.weight, nonlinearity='relu'); # kaiming with relu is equivalent to he_normal in keras
#     bn = nn.BatchNorm2d(out_channels);
#     return conv, bn;

## define ACDNet Fundamental Structures

In [3]:
class ACDNetV2(nn.Module):
    def __init__(self, input_length, n_class, sr, ch_conf=None):
        super(ACDNetV2, self).__init__();
        self.input_length = input_length;
        self.ch_config = ch_conf;

        stride1 = 2;
        stride2 = 2;
        channels = 8;
        k_size = (3, 3);
        n_frames = (sr/1000)*10; #No of frames per 10ms

        sfeb_pool_size = int(n_frames/(stride1*stride2));
        # tfeb_pool_size = (2,2);
        if self.ch_config is None:
            self.ch_config = [channels, channels*8, channels*4, channels*8, channels*8, channels*16, channels*16, channels*32, channels*32, channels*64, channels*64, n_class];
        # avg_pool_kernel_size = (1,4) if self.ch_config[1] < 64 else (2,4);
        fcn_no_of_inputs = self.ch_config[-1];
        conv1, bn1 = self.make_layers(1, self.ch_config[0], (1, 9), (1, stride1));
        conv2, bn2 = self.make_layers(self.ch_config[0], self.ch_config[1], (1, 5), (1, stride2));
        conv3, bn3 = self.make_layers(1, self.ch_config[2], k_size, padding=1);
        conv4, bn4 = self.make_layers(self.ch_config[2], self.ch_config[3], k_size, padding=1);
        conv5, bn5 = self.make_layers(self.ch_config[3], self.ch_config[4], k_size, padding=1);
        conv6, bn6 = self.make_layers(self.ch_config[4], self.ch_config[5], k_size, padding=1);
        conv7, bn7 = self.make_layers(self.ch_config[5], self.ch_config[6], k_size, padding=1);
        conv8, bn8 = self.make_layers(self.ch_config[6], self.ch_config[7], k_size, padding=1);
        conv9, bn9 = self.make_layers(self.ch_config[7], self.ch_config[8], k_size, padding=1);
        conv10, bn10 = self.make_layers(self.ch_config[8], self.ch_config[9], k_size, padding=1);
        conv11, bn11 = self.make_layers(self.ch_config[9], self.ch_config[10], k_size, padding=1);
        conv12, bn12 = self.make_layers(self.ch_config[10], self.ch_config[11], (1, 1));
        fcn = nn.Linear(fcn_no_of_inputs, n_class);
        nn.init.kaiming_normal_(fcn.weight, nonlinearity='sigmoid') # kaiming with sigoid is equivalent to lecun_normal in keras

        self.sfeb = nn.Sequential(
            #Start: Filter bank
            conv1, bn1, nn.ReLU(),\
            conv2, bn2, nn.ReLU(),\
            nn.MaxPool2d(kernel_size=(1, sfeb_pool_size))
        );

        tfeb_modules = [];
        self.tfeb_width = int(((self.input_length / sr)*1000)/10); # 10ms frames of audio length in seconds
        tfeb_pool_sizes = self.get_tfeb_pool_sizes(self.ch_config[1], self.tfeb_width);
        p_index = 0;
        for i in [3,4,6,8,10]:
            tfeb_modules.extend([eval('conv{}'.format(i)), eval('bn{}'.format(i)), nn.ReLU()]);

            if i != 3:
                tfeb_modules.extend([eval('conv{}'.format(i+1)), eval('bn{}'.format(i+1)), nn.ReLU()]);

            h, w = tfeb_pool_sizes[p_index];
            if h>1 or w>1:
                tfeb_modules.append(nn.MaxPool2d(kernel_size = (h,w)));
            p_index += 1;

        tfeb_modules.append(nn.Dropout(0.2));
        tfeb_modules.extend([conv12, bn12, nn.ReLU()]);
        h, w = tfeb_pool_sizes[-1];
        if h>1 or w>1:
            tfeb_modules.append(nn.AvgPool2d(kernel_size = (h,w)));
        tfeb_modules.extend([nn.Flatten(), fcn]);

        self.tfeb = nn.Sequential(*tfeb_modules);

        self.output = nn.Sequential(
            nn.Softmax(dim=1)
        );
        

    def forward(self, x):
        x = self.sfeb(x);
        #swapaxes
        x = x.permute((0, 2, 1, 3));
        x = self.tfeb(x);
        y = self.output[0](x);
        return y;

    def make_layers(self, in_channels, out_channels, kernel_size, stride=(1,1), padding=0, bias=False):
        conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias);
        nn.init.kaiming_normal_(conv.weight, nonlinearity='relu'); # kaiming with relu is equivalent to he_normal in keras
        bn = nn.BatchNorm2d(out_channels);
        return conv, bn;

    def get_tfeb_pool_sizes(self, con2_ch, width):
        h = self.get_tfeb_pool_size_component(con2_ch);
        w = self.get_tfeb_pool_size_component(width);
        # print(w);
        pool_size = [];
        for  (h1, w1) in zip(h, w):
            pool_size.append((h1, w1));
        return pool_size;

    def get_tfeb_pool_size_component(self, length):
        # print(length);
        c = [];
        index = 1;
        while index <= 6:
            if length >= 2:
                if index == 6:
                    c.append(length);
                else:
                    c.append(2);
                    length = length // 2;
            else:
               c.append(1);

            index += 1;

        return c;
#GetACDNetModel中的nclass不能改50以外的值，因為pretrain model本來的分類為50，載入後，下面的函式會再改動後面的分類。
def GetACDNetModel(input_len=30225, nclass=50, sr=20000, channel_config=None):
    net = ACDNetV2(input_len, nclass, sr, ch_conf=channel_config);
    return net;

In [5]:
net  = GetACDNetModel(input_len=one_sec_input_len, nclass=4, sr=20000, channel_config=None);

In [7]:
# calc.summary(net, (1,1,one_sec_input_len));

## Data Generator

In [8]:
class TLGenerator():
    #Generates data for Keras
    def __init__(self, samples=None, labels=None, options=None, classes_dict=None):
        random.seed(42);
        #Initialization
        # print(f"length of samples:{len(samples)}")
        self.data = [(samples[i], labels[i]) for i in range (0, len(samples))];
        self.opt = options;
        self.batch_size = options.batchSize;
        self.preprocess_funcs = self.preprocess_setup();
        self.mapdict = classes_dict

    def __len__(self):
        #Denotes the number of batches per epoch
        return int(np.floor(len(self.data) / self.batch_size));
        #return len(self.samples);

    def __getitem__(self, batchIndex):
        #Generate one batch of data
        batchX, batchY = self.generate_batch(batchIndex);
        # batchX, batchY = self.generate_batch_select_fixed_class(batchIndex)
        batchX = np.expand_dims(batchX, axis=1);
        batchX = np.expand_dims(batchX, axis=3);
        return batchX, batchY

    def generate_batch(self, batchIndex):
        #Generates data containing batch_size samples
        sounds = [];
        labels = [];
        indexes = None;
        
        for i in range(self.batch_size):
            # Training phase of BC learning
            # Select two training examples
            while True:
                sound1, label1 = self.data[random.randint(0, len(self.data) - 1)]
                sound2, label2 = self.data[random.randint(0, len(self.data) - 1)]
                if label1 != label2:
                    break
            sound1 = self.preprocess(sound1)
            sound2 = self.preprocess(sound2)

            # Mix two examples
            r = np.array(random.random())
            sound = U.mix(sound1, sound2, r, self.opt.sr).astype(np.float32)
            # print(f"sound length after U.mix is {len(sound)}")
            # print(f"nClasses:{self.opt.nClasses}, type of mapdict:{type(self.mapdict)}, type of label1:{type(label1)}")
            eye = np.eye(self.opt.nClasses)
            idx1 = self.mapdict[str(label1)]- 1
            idx2 = self.mapdict[str(label2)] - 1
            label = (eye[idx1] * r + eye[idx2] * (1 - r)).astype(np.float32)
            # label = (eye[label1] * r + eye[label2] * (1 - r)).astype(np.float32)

            #For stronger augmentation
            sound = U.random_gain(6)(sound).astype(np.float32)
            # print(f"sound length after U.random_gain is {len(sound)}")
            sounds.append(sound);
            labels.append(label);

        sounds = np.asarray(sounds);
        labels = np.asarray(labels);
        # print(f"batchIndex is {batchIndex}, total sounds is {len(sounds)}")
        # print(f"labels in generate_batch is:\n{labels}")
        return sounds, labels;

    def generate_batch_select_fixed_class(self, batchIndex):
        #Generates data containing batch_size samples
        sounds = [];
        labels = [];
        indexes = None;
        #two variables recording alarm and moaning sounds count
        alarm_selected = 0;
        moaning_selected = 0;
        help_eng_selected = 0;
        for i in range(self.batch_size):
            # Training phase of BC learning
            # Select two training examples
            while True:
                # print("enter while true")
                sound1, label1 = self.data[random.randint(0, len(self.data) - 1)]
                sound2, label2 = self.data[random.randint(0, len(self.data) - 1)]
                lbl1_int = np.int16(label1);
                lbl2_int = np.int16(label2);
                # print(f"label1:{label1} and label2:{label2}")
                # print(f"label1:{type(label1)} and label2:{type(label2)}")
                if label1 != label2:
                    # print("enter first layer if");
                    if (lbl1_int == 52 and lbl2_int == 99) or (lbl1_int == 99 and lbl2_int ==52):
                        # print("enter 52 second layer if");
                        # if (alarm_selected < moaning_selected) or (alarm_selected == moaning_selected):
                        if (alarm_selected == moaning_selected) and (alarm_selected == help_eng_selected):
                            # print("enter 52 third layer if");
                            alarm_selected += 1;
                            # print(f"alarm sound selected, label1:{label1}, label2:{label2}, alarm_selected:{alarm_selected}, moaning_selected:{moaning_selected}");
                            # print("perform break at 52");
                            break;
                    if (lbl1_int == 56 and lbl2_int == 99) or (lbl1_int == 99 and lbl2_int == 56):
                        # print("enter 56 second layer if");
                        # if (moaning_selected < alarm_selected) or (alarm_selected == moaning_selected):
                        if (moaning_selected < alarm_selected) and (moaning_selected == help_eng_selected):
                            # print("enter 56 third layer if");
                            moaning_selected += 1;
                            # print(f"alarm sound selected, label1:{label1}, label2:{label2}, alarm_selected:{alarm_selected}, moaning_selected:{moaning_selected}");
                            # print("perform break at 56");
                            break;
                    if (lbl1_int == 71 and lbl2_int == 99) or (lbl1_int == 99 and lbl2_int == 71):
                        # print("enter 56 second layer if");
                        if (help_eng_selected < alarm_selected) and (help_eng_selected < moaning_selected):
                            # print("enter 56 third layer if");
                            moaning_selected += 1;
                            # print(f"alarm sound selected, label1:{label1}, label2:{label2}, alarm_selected:{alarm_selected}, moaning_selected:{moaning_selected}");
                            # print("perform break at 56");
                            break;
            # print(f"escape for loop");
            sound1 = self.preprocess(sound1)
            sound2 = self.preprocess(sound2)

            # Mix two examples
            r = np.array(random.random());
            #######make wanted class mix ration above 0.5##########
            # iLbl1 = np.int16(label1);
            # iLbl2 = np.int16(label2);
            # r = 1.0;
            # p_ratio1 = 0.4
            # p_ratio2 = 0.6
            # while True:
            #     r = np.array(random.random());
            #     if r > p_ratio1 and iLbl1 != 99 :
            #         break;
            #     if r < p_ratio2 and iLbl2 != 99 :
            #         break;
            #######################End#######################
            # print(f"r:{r}, lbl1:{label1}, lbl2:{label2}")  
            sound = U.mix(sound1, sound2, r, self.opt.sr).astype(np.float32)
            eye = np.eye(self.opt.nClasses)
            idx1 = self.mapdict[str(label1)]- 1
            idx2 = self.mapdict[str(label2)] - 1
            label = (eye[idx1] * r + eye[idx2] * (1 - r)).astype(np.float32)
            

            #For stronger augmentation
            sound = U.random_gain(6)(sound).astype(np.float32)
            # print(f"sound length after U.random_gain is {len(sound)}")
            sounds.append(sound);
            labels.append(label);

        sounds = np.asarray(sounds);
        labels = np.asarray(labels);
        # print(f"batchIndex is {batchIndex}, total sounds is {len(sounds)}")
        # print(f"labels in generate_batch is:\n{labels}")
        # print(f"alarm_selected:{alarm_selected}, moaning_selected:{moaning_selected}");
        return sounds, labels;
    
    def preprocess_setup(self):
        funcs = []
        if self.opt.strongAugment:
            funcs += [U.random_scale(1.25)]

        funcs += [U.padding(self.opt.inputLength // 2),
                  U.random_crop(self.opt.inputLength),
                  U.normalize(32768.0)]
        return funcs

    def preprocess(self, sound):
        for f in self.preprocess_funcs:
            sound = f(sound)

        return sound;